# 2.3 SOO for 2D vector field

In [ ]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

We load the necessary objects.

In [ ]:
from run_me import *


# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = PlotManager()
true_temp_field = pickle_manager.read_file('simulation', 'disp_plane_field.obj')
grid = pickle_manager.read_file('simulation', 'disp_plane_points.obj')

field_bounds = true_temp_field.get_bounds()
sensor_bounds = field_bounds+np.array([[1, 1], [-1, -1]])*0.002

We setup the sensor suite

In [ ]:
# Setup the symmetry
symmetry_manager = SymmetryManager()
symmetry_manager.set_2D_x(np.mean(field_bounds[:, 0]))

# Setup the sensor suite
def f(x): return np.zeros(x.shape)
sensor = PointSensor(0, f, 0, np.array([[-5e10, -5e10, -5e10], [5e10, 5e10, 5e10]]), 2)
sensors = np.array([sensor]*5)

sensor_suite = SensorSuite(
    VectorField(RBFModel, field_bounds), 
    sensors,
    symmetry=[symmetry_manager.reflect_2D_horiz]
)

We setup the optimisation.

In [ ]:
# Setup the experiment
optimiser = PSOOptimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)
experiment.plan_soo(
    sensor_suite,
    sensor_bounds
)
res = experiment.design()
proposed_layout, true_disps, model_disps, sensor_vals = experiment.get_SOO_plotting_arrays(res.X)

We plot the mangitudes of the fields to compare.

In [ ]:
mag_true_disps = np.linalg.norm(true_disps, axis=1).reshape(-1, 1)
mag_model_disps = np.linalg.norm(model_disps, axis=1).reshape(-1, 1)
mag_sensor_vals = np.linalg.norm(sensor_vals, axis=1).reshape(-1, 1)

# Display the results
graph_manager.build_optimisation(
    res.history
)
graph_manager.draw()
graph_manager.build_2D_compare(
    grid,
    proposed_layout,
    mag_true_disps,
    mag_model_disps
)
graph_manager.draw()
graph_manager.build_3D_compare(
    grid,
    mag_true_disps,
    mag_model_disps
)
graph_manager.draw()